Ответить на вопросы по датасету.

**Датасет**

Вопросы:

- Каков процент женщин в датасете?
- Каков процент мужчин в датасете?
- Кто самый пожилой из расстрелянных? (ФИО и возраст)
- Кто самый молодой из расстрелянных? (ФИО и возраст)
- Каков средний возраст расстрелянных?
- Где больше всего захоронений репрессированных? (с указанием количества захоронений)
- Каков процент русских среди расстрелянных?
- Каков процент евреев среди расстрелянных?

In [138]:
import pandas as pd
import numpy as np
import re

In [139]:
df=pd.read_csv('shot_by_soviets.csv')

In [140]:
df=df.drop('ADDRESS_ID', axis=1).drop('APT', axis=1).drop('LINK', axis=1)

In [141]:
def b_year(row):
    list_ = [
        r'род. ([0-9]{4})',
        r'род. [0-9]{1,2}\.[0-9]{1,2}\.([0-9]{4})',
        r'род. в \w+ ([0-9]{4})'
    ]
    for year_str in list_:
        result = re.findall(year_str, row)
        if result:
            return result[0]
    return 0

In [142]:
df['b_year'] = df['BIO'].apply(b_year)
df['b_year'] = df['b_year'].astype('int64')

In [143]:
def d_year(row):
    result = re.findall(r'Рассстрел\w+ [0-9]{1,2}\.[0-9]{1,2}\.([0-9]{4})', row)
    if result:
        return result[0]
    return 0

In [144]:
df['d_year'] = df['BIO'].apply(d_year)
df['d_year'] = df['d_year'].astype('int64')
df['age'] = df['d_year'] - df['b_year']
df['age'] = df['age'].replace(1939, 0).replace(1937, 0)

In [145]:
nat=[r'абхазец\W', r'австриец\W', r'австрийка\W', r'азербайджанец\W', r'англичанин\W', r'араб\W', r'армянин\W', r'армянка\W', r'ассириец\W', r'афганец\W', r'башкир\W', r'башкирка\W', 
r'белорус\W', r'белоруска\W', r'болгарин\W', r'бурят\W', r'венгерка\W', r'венгр\W', r'голландец\W', r'грек\W', r'грузин\W', r'грузинка\W', r'даргинец\W', r'датчанин\W', r'еврей\W', r'еврейка\W', r'индус\W', 
r'иранец\W', r'итальянец\W', r'казах\W', r'калмык\W', r'караим\W', r'караимка\W', r'карачаевец\W', r'карел\W', r'китаец\W', r'китаянка\W', r'кореец\W', r'кореянка\W', r'кумык\W', r'латгалец\W', r'латыш\W',
r'латышка\W', r'литовец\W', r'литовка\W', r'мадьяр\W', r'мариец\W', r'молдаванин\W', r'монгол\W', r'мордвин\W', r'немец\W', r'немка\W', r'ойрат\W', r'осетин\W', r'перс\W', r'полька\W', r'поляк\W', r'румын\W',
r'румынка\W', r'русская\W', r'русский\W', r'серб\W', r'словенец\W', r'таджик\W', r'татарин\W', r'татарка\W', r'турок\W', r'тюрок\W', r'удмурт\W', r'узбек\W', r'украинец\W', r'украинка\W', 
r'финка\W', r'финн\W', r'француз\W', r'хорват\W', r'черногорец\W', r'чех\W', r'чешка\W', r'чуваш\W', r'швед\W', r'шведка\W', r'швейцар\W', r'швейцарец\W', r'эстонец\W', r'эстонка\W', r'якут\W',
r'японец\W', r'японка\W']

In [146]:
def nation(row):
    for n in nat:
        result = re.findall(n, row)
        if result:
            return result[0].strip(',').strip().strip(')').strip('.')
    return '-'

In [ ]:
df['nationality'] = df['BIO'].apply(nation)
df['nationality'].unique()

In [148]:
def gender(row):
    result = re.findall(r'(?=ка)\w+', row)
    if result:
        return 'ж'
    return 'м'

In [149]:
df['gender'] = df['nationality'].apply(gender)

In [150]:
def burial(row):
    result = re.findall(r'(?<=захоронения: )\w+', row)
    if result:
        return result[0]
    return 0

In [151]:
df['burial'] = df['BIO'].apply(burial)

- Каков процент женщин в датасете?
- Каков процент мужчин в датасете?

In [158]:
df['gender'].value_counts()

м    11219
ж      570
Name: gender, dtype: int64

In [159]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11789 entries, 0 to 11788
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   NAME         11789 non-null  object
 1   BIO          11789 non-null  object
 2   b_year       11789 non-null  int64 
 3   d_year       11789 non-null  int64 
 4   age          11789 non-null  int64 
 5   nationality  11789 non-null  object
 6   gender       11789 non-null  object
 7   burial       11789 non-null  object
dtypes: int64(3), object(5)
memory usage: 736.9+ KB


In [163]:
print(f'% женщин: {round((570/11789)*100, 1)}')
print(f'% мужчин: {round((11219/11789)*100, 1)}')

% женщин: 4.8
% мужчин: 95.2


- Кто самый пожилой из расстрелянных? (ФИО и возраст)
- Кто самый молодой из расстрелянных? (ФИО и возраст)

In [156]:
df['age'].max()
df[df['age']==81]

,NAME,BIO,b_year,d_year,age,nationality,gender,burial
2317,Фукс Альберт Львович,"род. 1857, г. Гродно, еврей, б/п, без определе...",1857,1938,81,еврей,м,Коммунарка
5740,Эглит Роберт Андреевич,"род. 1857, г. Рига, латыш, член ВКП(б), зам. н...",1857,1938,81,латыш,м,Коммунарка


In [153]:
df['age'].min()
df[df['age']==16]

,NAME,BIO,b_year,d_year,age,nationality,gender,burial
749,Гельберт Абрам Григорьевич,"род. 1921, в Иркутске, еврей, б/п, без определ...",1921,1937,16,еврей,м,Донское


- Каков средний возраст расстрелянных?

In [157]:
df['age'].mean()

43.18602086690983

- Каков процент русских среди расстрелянных?
- Каков процент евреев среди расстрелянных?

In [170]:
df[(df['nationality']=='русский') | (df['nationality']=='русская')]

,NAME,BIO,b_year,d_year,age,nationality,gender,burial
1,Кузьмин Василий Иванович,"род. 1904, Самарская губ., Бузулукский уезд, д...",1904,1938,34,русский,м,Бутово
2,Чернышев Илья Константинович,"род. 1907, г. Харбина (Северный Китай), русски...",1907,1937,30,русский,м,Бутово
3,Колесникова Ксения Васильевна,"род. 1899, Пензенская губ., Саранский уезд, д....",1899,1937,38,русская,ж,Бутово
4,Колесников Иван Михайлович,"род. 1895, Воронежская обл., Новохоперский р-н...",1895,1938,43,русский,м,Бутово
5,Хворов Андрей Дмитриевич,"род. 1886, г. Ташкент, русский, член ВКП(б), з...",1886,1938,52,русский,м,Бутово
...,...,...,...,...,...,...,...,...
11775,Максаков Яков Григорьевич,"род. 1882, Западная обл., Сафоновский р-н, с. ...",1882,1937,55,русский,м,Бутово
11777,Новиков Михаил Никитович,"род. 1906, в д.Озерово Мосальского уезда Калуж...",1906,1939,33,русский,м,Донское
11778,Жильцов Михаил Иванович,"род. 1915, г. МОсквы, русский, б/п, завод № 20...",1915,1937,22,русский,м,Бутово
11781,Стрелков Аркадий Александрович,"род. 1896, Тамбовская губ., г. Моршанск, русск...",1896,1938,42,русский,м,Коммунарка


In [171]:
df[(df['nationality']=='еврей') | (df['nationality']=='еврейка')]

,NAME,BIO,b_year,d_year,age,nationality,gender,burial
9,Глянц Рахмиль Моисеевич,"род. 1910, Польша, г. Замостье, еврей, б/п, Ак...",1910,1937,27,еврей,м,Бутово
14,Юзефович Иосиф Самойлович,"род. 1904, Украинская ССР, г. Полтава, еврей, ...",1904,1941,37,еврей,м,Коммунарка
15,Рицкий Илья Ильич,"род. 1893, г. Варшава, еврей, б/п, технический...",1893,1938,45,еврей,м,Коммунарка
20,Пер Шая Мотхелевич,"род. 1901, Латвия, г. Рига, еврей, член ВКП(б)...",1901,1938,37,еврей,м,Бутово
22,Маркович Борис Владимирович,"род. 1901, Румыния, г. Кишинев, еврей, член ВК...",1901,1938,37,еврей,м,Бутово
...,...,...,...,...,...,...,...,...
11748,Лурье Моисей Ильич (Эмель Александр),"род. 1897, в д.Косаричи Бобруйского уезда Минс...",1897,1936,39,еврей,м,Донское
11754,Фрадкин Николай Ильич,"род. 1905, Украинская ССР, Черниговская обл., ...",1905,1938,33,еврей,м,Бутово
11776,Юзефович (Шпинак) Иосиф Сигизмундович,"род. 1890, в Варшаве, еврей, член ВКП(б), млад...",1890,1952,62,еврей,м,Донское
11780,Аронов Зигмунд-Зусман Борисович,"род. 1906, г. Варшава, еврей, бывший член ВКП(...",1906,1937,31,еврей,м,Бутово


In [172]:
print(f'% русских: {round((5144/11789)*100, 1)}')
print(f'% евреев: {round((2216/11789)*100, 1)}')

% русских: 43.6
% евреев: 18.8


- Где больше всего захоронений репрессированных? (с указанием количества захоронений)

In [154]:
df['burial'].value_counts()

Бутово           5048
Коммунарка       3675
Донское          2465
Ваганьковское     549
Яузская            52
Name: burial, dtype: int64